In [1]:
import subprocess
import os

In [2]:
!dx login --token #[token]


Note: Use dx select --level VIEW or dx select --public to
select from projects for which you only have VIEW permissions.

Available projects (CONTRIBUTE or higher):
Setting current project to: Exome full cohort - Jan 05, 2023


In [3]:
!dx pwd

Exome full cohort - Jan 05, 2023:/


In [4]:
inpath_dnanexus = '/Regenie_infolder/'
outpath_dnanexus = '/Regenie_outfolder/'

os.system(f'dx mkdir {inpath_dnanexus}')
os.system(f'dx mkdir {outpath_dnanexus}')

0

In [ ]:
#Upload Step 1 files
outpath = '/Regenie_step1/Output'
file_list = os.listdir(outpath)
file_list = [f for f in file_list if f.endswith('loco')]
file_list = [f for f in file_list if 'GrCh38' in f]
for f in file_list:
    os.system(f'dx upload {outpath}/{f} --destination {inpath_dnanexus}')

In [ ]:
#Upload pred.list with correct dirname for DNAnexus
outpath = '/Regenie_step1/Output'
file_list = os.listdir(outpath)
file_list = [f for f in file_list if f.endswith('pred.list')]
file_list = [f for f in file_list if 'GrCh38' in f]
print(file_list)
os.system(f"cp {outpath}/{file_list[0]} {outpath}/{file_list[0].replace('GrCh38_pred.list', 'GrCh38_corrected_path_pred.list')}") 

os.system(f"sed -i 's@{outpath}/@''@g' {outpath}/{file_list[0].replace('GrCh38_pred.list', 'GrCh38_corrected_path_pred.list')}")
#Upload this pred.list with corrected path 
os.system(f"dx upload {outpath}/{file_list[0].replace('GrCh38_pred.list', 'GrCh38_corrected_path_pred.list')} --destination {inpath_dnanexus}")

In [9]:
!dx ls /Regenie_infolder/

CMR_phenotypes_regenie_15112023.txt
UKB_step1_commonvariant_GrCh38_1.loco
UKB_step1_commonvariant_GrCh38_10.loco
UKB_step1_commonvariant_GrCh38_11.loco
UKB_step1_commonvariant_GrCh38_12.loco
UKB_step1_commonvariant_GrCh38_2.loco
UKB_step1_commonvariant_GrCh38_3.loco
UKB_step1_commonvariant_GrCh38_4.loco
UKB_step1_commonvariant_GrCh38_5.loco
UKB_step1_commonvariant_GrCh38_6.loco
UKB_step1_commonvariant_GrCh38_7.loco
UKB_step1_commonvariant_GrCh38_8.loco
UKB_step1_commonvariant_GrCh38_9.loco
UKB_step1_commonvariant_GrCh38_corrected_path_pred.list


In [ ]:
%%bash

#Performing Regenie Step 2 with swiss-army-knife (repeat for RV with updated variables)
project="project-ID"
impute_file_dir="/Bulk/Imputation/Imputation from genotype (TOPmed)"
# Data field to TOPMed release
data_field="ukb21007"
inpath_dnanexus='/Regenie_infolder/'
outpath_dnanexus='/Regenie_outfolder/'

for i in {1..22};
do
    run_regenie_LV_GWAS="regenie --step 2 --bgen ${data_field}_c${i}_b0_v1.bgen \
      --ref-first \
      --sample ${data_field}_c${i}_b0_v1.sample \
      --phenoFile CMR_phenotypes_regenie_15112023.txt \
      --phenoCol LVEDV,LVESV,LVSV,LVEF,LVM,LVMVR,LVGFI,LVMCF,LVEDV_BSA,LVESV_BSA,LVSV_BSA,LVM_BSA  \
      --qt \
      --pred UKB_step1_commonvariant_GrCh38_corrected_path_pred.list  \
      --minMAC 20 \
      --bsize 1000 \
      --verbose \
      --debug \
      --out LV_commonvariant_chr${i}_TOPmed"

    dx run swiss-army-knife -iin="${impute_file_dir}/${data_field}_c${i}_b0_v1.bgen" \
     -iin="${impute_file_dir}/${data_field}_c${i}_b0_v1.bgen.bgi" \
     -iin="${impute_file_dir}/${data_field}_c${i}_b0_v1.sample" \
     -iin="${inpath_dnanexus}CMR_phenotypes_regenie_15112023.txt" \
     -iin="${inpath_dnanexus}UKB_step1_commonvariant_GrCh38_corrected_path_pred.list" \
     -iin="${inpath_dnanexus}UKB_step1_commonvariant_GrCh38_1.loco" \
     -iin="${inpath_dnanexus}UKB_step1_commonvariant_GrCh38_2.loco" \
     -iin="${inpath_dnanexus}UKB_step1_commonvariant_GrCh38_3.loco" \
     -iin="${inpath_dnanexus}UKB_step1_commonvariant_GrCh38_4.loco" \
     -iin="${inpath_dnanexus}UKB_step1_commonvariant_GrCh38_5.loco" \
     -iin="${inpath_dnanexus}UKB_step1_commonvariant_GrCh38_6.loco" \
     -iin="${inpath_dnanexus}UKB_step1_commonvariant_GrCh38_7.loco" \
     -iin="${inpath_dnanexus}UKB_step1_commonvariant_GrCh38_8.loco" \
     -iin="${inpath_dnanexus}UKB_step1_commonvariant_GrCh38_9.loco" \
     -iin="${inpath_dnanexus}UKB_step1_commonvariant_GrCh38_10.loco" \
     -iin="${inpath_dnanexus}UKB_step1_commonvariant_GrCh38_11.loco" \
     -iin="${inpath_dnanexus}UKB_step1_commonvariant_GrCh38_12.loco" \
     -icmd="${run_regenie_LV_GWAS}" --tag="LV_GWAS_Regenie_Step2" --instance-type "mem1_hdd1_v2_x16" \
     -imount_inputs=true \
     --destination="${project}:${outpath_dnanexus}" --verbose --yes
done
